<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250213%5D%20Implementing%20PGD%20Adversarial%20on%20UNSW-NB15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hybrid model With PGD adversarial training**

In [ ]:
!pip uninstall -y foolbox
!pip uninstall -y foolbox-native

In [ ]:
!pip install foolbox==3.3.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.3 MB/s eta 0:00:00


**🔹 Step 1: Data Preprocessing (Save to File)**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
#import foolbox as fb  # For CW and PGD attacks

In [ ]:
# --- Load and preprocess dataset ---
dataset = pd.read_csv("Dataset_10Classes.csv").dropna()
for column in dataset.columns:
    if dataset[column].dtype == 'object':
        dataset[column] = LabelEncoder().fit_transform(dataset[column])
X = dataset.drop(['Class'], axis=1)
y = dataset['Class']

In [ ]:
print("Original Class Distribution:", np.bincount(y))

Original Class Distribution: [93000  2677  2329 16353 44525 24246 58871 13987  1511   174]


In [ ]:
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Apply SMOTE for class balance
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)
# Verify the balanced dataset distribution
print("Balanced Class Distribution:", np.bincount(y))

Balanced Class Distribution: [93000 93000 93000 93000 93000 93000 93000 93000 93000 93000]


In [ ]:
import pickle

# ✅ Save preprocessed data
with open("preprocessed_data.pkl", "wb") as f:
    pickle.dump((X, y), f)

print("✅ Preprocessed data saved!")

✅ Preprocessed data saved!


In [ ]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# ✅ Load preprocessed data
with open("preprocessed_data.pkl", "rb") as f:
    X, y = pickle.load(f)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ✅ Train Logistic Regression for Feature Extraction
lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=500, random_state=42)
lr_model.fit(X_train, y_train)

# ✅ Extract probabilities for BiLSTM
train_lr_features = lr_model.predict_proba(X_train)
test_lr_features = lr_model.predict_proba(X_test)
train_lr_features = np.expand_dims(train_lr_features, axis=1)
test_lr_features = np.expand_dims(test_lr_features, axis=1)

# ✅ One-hot encode labels
num_classes = len(np.unique(y))
y_train_encoded = to_categorical(y_train, num_classes=num_classes)
y_test_encoded = to_categorical(y_test, num_classes=num_classes)

# ✅ Save features
with open("lr_features.pkl", "wb") as f:
    pickle.dump((train_lr_features, test_lr_features, y_train_encoded, y_test_encoded), f)

print("✅ Logistic Regression features saved!")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


✅ Logistic Regression features saved!


**🔹 Step 3: Hybrid LR-BiLSTM Training**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, Dense
from tensorflow.keras.regularizers import l2
import pickle

# ✅ Load Logistic Regression features
with open("lr_features.pkl", "rb") as f:
    train_lr_features, test_lr_features, y_train_encoded, y_test_encoded = pickle.load(f)

# ✅ Define BiLSTM Model
bilstm_model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.0001)), input_shape=(1, train_lr_features.shape[2])),
    Dropout(0.2),
    Bidirectional(LSTM(32, kernel_regularizer=l2(0.0001))),
    Dropout(0.2),
    Dense(y_train_encoded.shape[1], activation="softmax")
])
bilstm_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# ✅ Train model
bilstm_model.fit(train_lr_features, y_train_encoded, validation_split=0.2, epochs=25, batch_size=64, verbose=1)

# ✅ Save trained model
bilstm_model.save("bilstm_model.h5")
print("✅ BiLSTM Model saved!")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/25
9300/9300 ━━━━━━━━━━━━━━━━━━━━ 50s 5ms/step - accuracy: 0.9925 - loss: 0.1702 - val_accuracy: 0.9999 - val_loss: 0.0035
Epoch 2/25
9300/9300 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 0.9999 - loss: 0.0031 - val_accuracy: 0.9999 - val_loss: 0.0017
Epoch 3/25
9300/9300 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 0.9999 - val_loss: 0.0013
Epoch 4/25
9300/9300 ━━━━━━━━━━━━━━━━━━━━ 43s 5ms/step - accuracy: 0.9999 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 5/25
9300/9300 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 6/25
9300/9300 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.9999 - val_loss: 0.0013
Epoch 7/25
9300/9300 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 1.0000 - val_loss: 0.0010
Epoch 8/25
9300/9300 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 1.0000 - loss: 9

✅ BiLSTM Model saved!


**🔸 PGD & FGSM Adversarial Training**

In [ ]:
import matplotlib.pyplot as plt
import foolbox as fb
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# ✅ Load the trained BiLSTM model
bilstm_model = tf.keras.models.load_model("bilstm_model.h5")

# ✅ Convert model to Foolbox format
fmodel = fb.TensorFlowModel(bilstm_model, bounds=(0, 1))

# ✅ Generate PGD Adversarial Examples (for evaluation)
def generate_pgd_examples(attack, X_test, y_test, eps=0.02, steps=10, batch_size=1024):
    """
    Generate PGD adversarial examples in batches for efficient computation.
    """
    num_samples = X_test.shape[0]
    adv_examples = []

    for i in range(0, num_samples, batch_size):
        batch_X = X_test[i:i + batch_size]
        batch_y = np.argmax(y_test[i:i + batch_size], axis=1)

        # Convert batch to TensorFlow tensor
        batch_X_tf = tf.convert_to_tensor(batch_X, dtype=tf.float32)

        # Generate adversarial examples using PGD
        adv_batch, *_ = attack(fmodel, batch_X_tf, batch_y, epsilons=eps)
        adv_examples.append(adv_batch.numpy())

    return np.vstack(adv_examples)

# ✅ Define PGD Attack (More Steps for Stronger Perturbation)
attack_pgd = fb.attacks.LinfPGD(steps=10)

# ✅ Generate Adversarial PGD Samples
print("\n🔹 Generating PGD Adversarial Examples for Model Evaluation...")
adv_pgd_test = generate_pgd_examples(attack_pgd, test_lr_features, y_test_encoded, eps=0.02, steps=10)

# ✅ Evaluate the model against PGD adversarial examples
print("\n🔹 Evaluating Model Performance on PGD Attacks...")
pgd_predictions = bilstm_model.predict(adv_pgd_test)
pgd_predictions_labels = np.argmax(pgd_predictions, axis=1)
true_labels = np.argmax(y_test_encoded, axis=1)

# ✅ Compute Performance Metrics
pgd_accuracy = accuracy_score(true_labels, pgd_predictions_labels)
pgd_precision = precision_score(true_labels, pgd_predictions_labels, average='macro')
pgd_recall = recall_score(true_labels, pgd_predictions_labels, average='macro')
pgd_f1 = f1_score(true_labels, pgd_predictions_labels, average='macro')

print("\n🔹 Hybrid Model Performance Against PGD Attacks:")
print("✅ Accuracy:", pgd_accuracy)
print("✅ Precision:", pgd_precision)
print("✅ Recall:", pgd_recall)
print("✅ F1-Score:", pgd_f1)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.



🔹 Generating PGD Adversarial Examples for Model Evaluation...

🔹 Evaluating Model Performance on PGD Attacks...
5813/5813 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step

🔹 Hybrid Model Performance Against PGD Attacks:
✅ Accuracy: 0.9999139784946236
✅ Precision: 0.9999140044997159
✅ Recall: 0.9999139784946237
✅ F1-Score: 0.999913973286192


In [ ]:
# prompt: Display detection rate (DR) of the Hybrid Model Performance Against PGD Attacks:

# Detection Rate (Recall for each class) against PGD attacks
pgd_detection_rate = recall_score(true_labels, pgd_predictions_labels, average=None)
for i in range(len(pgd_detection_rate)):
    print(f"Detection Rate for Class {i} against PGD attacks: {pgd_detection_rate[i]:.10f}")


Detection Rate for Class 0 against PGD attacks: 1.0000000000
Detection Rate for Class 1 against PGD attacks: 1.0000000000
Detection Rate for Class 2 against PGD attacks: 0.9999462366
Detection Rate for Class 3 against PGD attacks: 1.0000000000
Detection Rate for Class 4 against PGD attacks: 1.0000000000
Detection Rate for Class 5 against PGD attacks: 0.9998924731
Detection Rate for Class 6 against PGD attacks: 0.9993010753
Detection Rate for Class 7 against PGD attacks: 1.0000000000
Detection Rate for Class 8 against PGD attacks: 1.0000000000
Detection Rate for Class 9 against PGD attacks: 1.0000000000
